# Set up

https://www.gurobi.com/documentation/9.5/refman/py_python_api_overview.html

In [87]:
# Libraries
import gurobipy as grb
from gurobipy import GRB
model = grb.Model("Model")

# Decision variables

https://www.gurobi.com/documentation/9.5/refman/py_model_addvar.html#pythonmethod:Model.addVar
https://www.gurobi.com/documentation/9.5/refman/py_model_addvars.html#pythonmethod:Model.addVars
https://www.gurobi.com/documentation/9.5/refman/py_model_addmvar.html#pythonmethod:Model.addMVar

In [93]:
# Add decision variables
# We take the same notation that the "elements de solution folder"
p = 5     # nb de personnel
n = 5     # nb de projet
q = 3     # nb de qualification
h = 12    # horizon temporel, nb de jour de projets

# Données personnelles
Q = [ [0,1], [2], [0], [1], [0,1,2]] #competences par personne
V = [ [0, 5, 8], [], [], [], []] #congés par personne

# Données des projets
G =  [10, 10, 10, 10, 10] #gain par projet
P =  [1, 1, 1, 1, 1] #penalite par projet
# nombre de jours par compétence par projet / ex : le projet 2 necessite 7 jours de travail sur la compétance C
C = [[4,3,2],[3,2,7],[4,1,5],[3,3,3],[1,1,4]] 
D = [3,5,8,9,11] # Deadline

# Variables
X = model.addVars(p, n, q, h, vtype=GRB.BINARY)     
Y = model.addVars(n, vtype=GRB.BINARY)  # Le projet est-il complété en fin d'horizon temporel ?
L = model.addVars(n, vtype=GRB.INTEGER) # Jours de retard de chaque projets
E = model.addVars(n, vtype=GRB.INTEGER) # Date de fin effective de realisation du projet

# Constraints

https://www.gurobi.com/documentation/9.5/refman/py_model_addconstr.html#pythonmethod:Model.addConstr
https://www.gurobi.com/documentation/9.5/refman/py_model_addconstrs.html#pythonmethod:Model.addConstrs

In [94]:
# Add constraints

no_ubiquity = []
for i in range(p):
    for t in range(h):
        no_ubiquity += [model.addConstr(sum(X[i, j, k, t] for j in range(n) for k in range(q)) <= 1)]

no_forced_work = []
for i in range(p):
    for t in V[i]:
        no_forced_work += [model.addConstr(sum(X[i, j, k, t] for j in range(n) for k in range(q)) == 0)]

end_of_poject = []
for j in range(n):
    for k in range(q):
        if C[j][k] > 0:
            end_of_poject += [model.addConstr( Y[j] * C[j][k] <= sum(X[i, j, k, t] for i in range(p) for t in range(h)))]

skill_constr = []
for i in range(p):
    for j in range(n):
        for k in range(q):
            for t in range(h):
                if (k not in Q[i]) or (C[j][k] == 0) :
                    skill_constr += [model.addConstr(X[i,j,k,t] == 0)]

no_overwork = []
for j in range(n):
    for k in range(q):
        no_overwork += [model.addConstr(sum(X[i, j, k, t] for i in range(p) for t in range(h)) <= C[j][k])]

model.addConstrs((X[i,j,k,t] * t <= E[j] for i in range(p) for j in range(n) for k in range(q) for t in range(h)))

model.addConstrs((E[j] - D[j] <= L[j] for j in range(n)))

model.update()

In [95]:
def remove_constr(constr_list):
    for constr in constr_list:
        model.remove(constr)

# remove_constr(no_ubiquity)
# remove_constr(no_forced_work)
# remove_constr(skill_constr)
# remove_constr(no_overwork)

# Optimization

https://www.gurobi.com/documentation/9.5/refman/py_lex.html#pythonclass:LinExpr

In [108]:
#model.setObjective(sum(Y[j] * G[j] - L[j] * P[j] for j in range(n)), GRB.MAXIMIZE)

#model.optimize()
#print("\nOptimal objective value: ", model.ObjVal)

In [113]:
T = model.addVars(n, vtype=GRB.INTEGER)  # durée du projet

for j in range(n):
    t_start = h        
    for t in range(h):
        for k in range(q):
            for i in range(p):
                if X[i,j,k,t].x > 0 :
                    if t<t_start:
                        t = t_start
                    else:
                        model.addConstr( t - t_start <= T[j] for j in range(n))

KeyError: (4, 0, 1, 12)

In [ ]:
for j in range(n):
    model.setObjective(T[j], GRB.MINIMIZE)

#model.optimize()
#print("\nOptimal objective value: ", model.ObjVal)

NameError: name 'T' is not defined

In [106]:
for i in range(p):
    jours_travaille = 0
    print("\nEmploi du temps personne ",i, " : ")
    for t in range(h):
        for k in range(q):
            for j in range(n):
                if X[i,j,k,t].x != 0:
                    jours_travaille += 1
                    print("Jour ", t, " Compétence ", k, " sur projet ", j)
    print("Jours travailés : ", jours_travaille)


Emploi du temps personne  0  : 
Jour  1  Compétence  0  sur projet  1
Jour  2  Compétence  0  sur projet  0
Jour  3  Compétence  0  sur projet  0
Jour  4  Compétence  1  sur projet  1
Jour  6  Compétence  1  sur projet  3
Jour  7  Compétence  0  sur projet  2
Jours travailés :  6

Emploi du temps personne  1  : 
Jour  0  Compétence  2  sur projet  1
Jour  1  Compétence  2  sur projet  1
Jour  2  Compétence  2  sur projet  1
Jour  3  Compétence  2  sur projet  1
Jour  4  Compétence  2  sur projet  2
Jour  5  Compétence  2  sur projet  1
Jour  6  Compétence  2  sur projet  2
Jour  7  Compétence  2  sur projet  3
Jour  8  Compétence  2  sur projet  2
Jour  9  Compétence  2  sur projet  4
Jour  10  Compétence  2  sur projet  4
Jours travailés :  11

Emploi du temps personne  2  : 
Jour  0  Compétence  0  sur projet  1
Jour  1  Compétence  0  sur projet  0
Jour  2  Compétence  0  sur projet  2
Jour  3  Compétence  0  sur projet  0
Jour  4  Compétence  0  sur projet  3
Jour  5  Compétence  

In [104]:
for j in range(n):
    print("\nAllocation pour le projet ",j, " : ")
    for k in range(q):
        for i in range(p):
            for t in range(h):
                if X[i,j,k,t].x != 0:
                    print(" Compétence ", k, " par personne ", i, " Jour ", t)


Allocation pour le projet  0  : 
 Compétence  0  par personne  0  Jour  2
 Compétence  0  par personne  0  Jour  3
 Compétence  0  par personne  2  Jour  1
 Compétence  0  par personne  2  Jour  3
 Compétence  1  par personne  3  Jour  0
 Compétence  1  par personne  3  Jour  1
 Compétence  1  par personne  3  Jour  2
 Compétence  2  par personne  4  Jour  0
 Compétence  2  par personne  4  Jour  2

Allocation pour le projet  1  : 
 Compétence  0  par personne  0  Jour  1
 Compétence  0  par personne  2  Jour  0
 Compétence  0  par personne  2  Jour  5
 Compétence  1  par personne  0  Jour  4
 Compétence  1  par personne  3  Jour  5
 Compétence  2  par personne  1  Jour  0
 Compétence  2  par personne  1  Jour  1
 Compétence  2  par personne  1  Jour  2
 Compétence  2  par personne  1  Jour  3
 Compétence  2  par personne  1  Jour  5
 Compétence  2  par personne  4  Jour  3
 Compétence  2  par personne  4  Jour  4

Allocation pour le projet  2  : 
 Compétence  0  par personne  0  Jour

# Objective function

https://www.gurobi.com/documentation/9.5/refman/py_model_setobjective.html#pythonmethod:Model.setObjective
https://www.gurobi.com/documentation/9.5/refman/py_model_setobjectiven.html#pythonmethod:Model.setObjectiveN

In [6]:
# Add objective function
# i.e. maximize the difference N_2 - N_1
# model.setObjective(N[2] - N[1], GRB.MAXIMIZE)

# Add multi-objective function
# i.e. minimize the sum M + N_1 + N_2 and then maximize the difference N_2 - N_1
# model.ModelSense = GRB.MINIMIZE
# model.setObjectiveN(total_sum, 0, 2)
# model.setObjectiveN(N[1] - N[2], 1, 1)

NameError: name 'N' is not defined

# Solving

https://www.gurobi.com/documentation/9.5/refman/py_model_optimize.html

In [ ]:
# Update model
model.update()

In [ ]:
# Display model
model.display()

Minimize
   <gurobi.LinExpr: M + N[1] + N[2]>
Subject To
   MVsDecimalDecomposition : <gurobi.LinExpr: M + -1.0 D[0] + -10.0 D[1] + -100.0 D[2] + -1000.0 D[3]> = 0.0
   SumDigits : <gurobi.LinExpr: D[0] + D[1] + D[2] + D[3]> = 12.0
   IncreasingDigits[0] : <gurobi.LinExpr: -1.0 D[0] + D[1]> <= -1.0
   IncreasingDigits[1] : <gurobi.LinExpr: -1.0 D[1] + D[2]> <= -1.0
   IncreasingDigits[2] : <gurobi.LinExpr: -1.0 D[2] + D[3]> <= -1.0
   NVsBinaryDecomposition[1] : <gurobi.LinExpr: N[1] + -1.0 B[1,0] + -2.0 B[1,1] + -4.0 B[1,2] + -8.0 B[1,3] + -16.0 B[1,4] + -32.0 B[1,5] + -64.0 B[1,6] + -128.0 B[1,7] + -256.0 B[1,8] + -512.0 B[1,9]> = 0.0
   NVsBinaryDecomposition[2] : <gurobi.LinExpr: N[2] + -1.0 B[2,0] + -2.0 B[2,1] + -4.0 B[2,2] + -8.0 B[2,3] + -16.0 B[2,4] + -32.0 B[2,5] + -64.0 B[2,6] + -128.0 B[2,7] + -256.0 B[2,8] + -512.0 B[2,9]> = 0.0
   NbNonZeroDigitsLB[1] : <gurobi.LinExpr: B[1,0] + B[1,1] + B[1,2] + B[1,3] + B[1,4] + B[1,5] + B[1,6] + B[1,7] + B[1,8] + B[1,9]> >= 2.0
   NbNo

In [ ]:
# Solving
# model.params.outputflag = 0 # Mute
# model.setParam('TimeLimit', 30) # Time limit in seconds
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 27 columns and 82 nonzeros
Model fingerprint: 0x9bcde240
Variable types: 0 continuous, 27 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolved: 11 rows and 24 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 () ...
----------

In [ ]:
# Get optimization status
print(model.Status == GRB.OPTIMAL, model.Status == GRB.TIME_LIMIT, model.Status == GRB.INFEASIBLE)

True False False


# Results

In [ ]:
# Get objective value
objective_value = model.objVal
print(objective_value)

258.0


https://www.gurobi.com/documentation/9.5/refman/py_model_getvarbyname.html

In [ ]:
# Get decision variables values
print(f"M={M.x}, N_1={N[1].x}, N_2={N[2].x}, B_(1,0)={B[1, 0].x}, B_(2,0)={B[2, 0].x}")
# or
# print(f"M={model.getVarByName('M').x}, \
#     N_1={model.getVarByName('N[1]').x}, N_2={model.getVarByName('N[2]').x}, \
#     B_(1,0)={model.getVarByName('B[1,0]').x}, B_(2,0)={model.getVarByName('B[2,0]').x}")

M=129.0, N_1=3.0, N_2=126.0, B_(1,0)=1.0, B_(2,0)=-0.0


In [ ]:
# Get linear expression values
print(total_sum.getValue())

258.0
